In [1]:
import nannyml as nml
from IPython.display import display

reference_df, analysis_df, analysis_target_df = nml.load_synthetic_car_loan_dataset()

analysis_df = analysis_df.merge(analysis_target_df, left_index=True, right_index=True)

display(reference_df.head(3))

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,repaid,timestamp,y_pred_proba,y_pred
0,39811.0,40K - 60K €,0.632950,19.0,False,40%,0.212653,1.0,2018-01-01 00:00:00.000,0.99,1
1,12679.0,40K - 60K €,0.718627,7.0,True,10%,4.927549,0.0,2018-01-01 00:08:43.152,0.07,0
2,19847.0,40K - 60K €,0.721724,17.0,False,0%,0.520817,1.0,2018-01-01 00:17:26.304,1.00,1


In [2]:
print(reference_df.head(3).to_markdown(tablefmt="grid"))

+----+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+-----------------+----------+-------------------------+----------------+----------+
|    |   car_value | salary_range   |   debt_to_income_ratio |   loan_length | repaid_loan_on_prev_car   | size_of_downpayment   |   driver_tenure |   repaid | timestamp               |   y_pred_proba |   y_pred |
+====+=============+================+========================+===============+===========================+=======================+=================+==========+=========================+================+==========+
|  0 |       39811 | 40K - 60K €    |               0.63295  |            19 | False                     | 40%                   |        0.212653 |        1 | 2018-01-01 00:00:00.000 |           0.99 |        1 |
+----+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+--------------

In [3]:
calc = nml.PerformanceCalculator(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    problem_type='classification_binary',
    metrics=['business_value'],
    business_value_matrix = [[5, -10],[-50, 50]],
    normalize_business_value='per_prediction',
    chunk_size=5000)

In [4]:
calc.fit(reference_df)

In [5]:
results = calc.calculate(analysis_df)
display(results.filter(period='analysis').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-10-30 18:00:00   
1    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
2  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
3  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
4  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
5  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
6  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
7  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
8  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
9  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                                         business_value  \
                 end_date    period targets_missing_rate sampling_error   
0 2018-11-30 00:27:16.848  analysis                  0.0       0.375491   
1 2018-12-30 07:03:16.848  analysis                  0.0       0.375491   
2 2019-01-29 13:39:16.848  analysis                  0.0       0.375491   
3 2019-02-28 20:15:16.848  analysis                  0.0       0.375491   
4 2019-03-31 02:51:16.848  analysis                  0.0       0.375491   
5 2019-04-30 09:27:16.848  analysis                  0.0       0.375491   
6 2019-05-30 16:03:16.848  analysis                  0.0       0.375491   
7 2019-06-29 22:39:16.848  analysis                  0.0       0.375491   
8 2019-07-30 05:15:16.848  analysis                  0.0       0.375491   
9 2019-08-29 11:51:16.848  analysis                  0.0       0.375491   

                                                  
    value upper_threshold lower_threshold  alert  
0  24.347       24.420349       22.826251  False  
1  23.076       24.420349       22.826251  False  
2  23.564       24.420349       22.826251  False  
3  23.330       24.420349       22.826251  False  
4  23.638       24.420349       22.826251  False  
5  22.163       24.420349       22.826251   True  
6  22.124       24.420349       22.826251   True  
7  22.061       24.420349       22.826251   True  
8  22.213       24.420349       22.826251   True  
9  22.617       24.420349       22.826251   True

In [6]:
from docs.utils import print_multi_index_markdown
print_multi_index_markdown(results.filter(period='analysis').to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+----------------------------+------------+--------------------------+--------------------+-----------+---------------------+---------------------+-----------+
|    | | chunk       |                 |                 |               |                     |                            |            |                          | | business_value   |           |                     |                     |           |
|    | | key         | | chunk_index   | | start_index   | | end_index   | | start_date        | | end_date                 | | period   | | targets_missing_rate   | | sampling_error   | | value   | | upper_threshold   | | lower_threshold   | | alert   |
+====+===============+=================+=================+===============+=====================+============================+============+==========================+====================+===========+=====================+===============

In [7]:
display(results.filter(period='reference').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-01-01 00:00:00   
1    [5000:9999]           1        5000      9999 2018-01-31 06:36:00   
2  [10000:14999]           2       10000     14999 2018-03-02 13:12:00   
3  [15000:19999]           3       15000     19999 2018-04-01 19:48:00   
4  [20000:24999]           4       20000     24999 2018-05-02 02:24:00   
5  [25000:29999]           5       25000     29999 2018-06-01 09:00:00   
6  [30000:34999]           6       30000     34999 2018-07-01 15:36:00   
7  [35000:39999]           7       35000     39999 2018-07-31 22:12:00   
8  [40000:44999]           8       40000     44999 2018-08-31 04:48:00   
9  [45000:49999]           9       45000     49999 2018-09-30 11:24:00   

                                                          business_value  \
                 end_date     period targets_missing_rate sampling_error   
0 2018-01-31 06:27:16.848  reference                  0.0       0.375491   
1 2018-03-02 13:03:16.848  reference                  0.0       0.375491   
2 2018-04-01 19:39:16.848  reference                  0.0       0.375491   
3 2018-05-02 02:15:16.848  reference                  0.0       0.375491   
4 2018-06-01 08:51:16.848  reference                  0.0       0.375491   
5 2018-07-01 15:27:16.848  reference                  0.0       0.375491   
6 2018-07-31 22:03:16.848  reference                  0.0       0.375491   
7 2018-08-31 04:39:16.848  reference                  0.0       0.375491   
8 2018-09-30 11:15:16.848  reference                  0.0       0.375491   
9 2018-10-30 17:51:16.848  reference                  0.0       0.375491   

                                                  
    value upper_threshold lower_threshold  alert  
0  23.483       24.420349       22.826251  False  
1  23.046       24.420349       22.826251  False  
2  24.028       24.420349       22.826251  False  
3  23.568       24.420349       22.826251  False  
4  23.514       24.420349       22.826251  False  
5  23.445       24.420349       22.826251  False  
6  23.715       24.420349       22.826251  False  
7  23.944       24.420349       22.826251  False  
8  23.772       24.420349       22.826251  False  
9  23.718       24.420349       22.826251  False

In [8]:
print_multi_index_markdown(results.filter(period='reference').to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+----------------------------+------------+--------------------------+--------------------+-----------+---------------------+---------------------+-----------+
|    | | chunk       |                 |                 |               |                     |                            |            |                          | | business_value   |           |                     |                     |           |
|    | | key         | | chunk_index   | | start_index   | | end_index   | | start_date        | | end_date                 | | period   | | targets_missing_rate   | | sampling_error   | | value   | | upper_threshold   | | lower_threshold   | | alert   |
+====+===============+=================+=================+===============+=====================+============================+============+==========================+====================+===========+=====================+===============

In [9]:
figure = results.plot()
figure.show()

In [10]:
figure = results.plot()
figure.write_image(f'../_static/tutorials/performance_calculation/binary/tutorial-business-value-calculation-binary-car-loan-analysis.svg')

# tutorial-perf-est-guide-binary-class-car-loan-analysis
